In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("disney").getOrCreate()

<h1>IMPORT DISNEY FILE </h1>

In [2]:
desdf = spark.read.option("header","true") \
.option("quote",'"') \
.option("escape","\\") \
.option("multiLine","true")\
.option("mode","PERMISSIVE")\
.csv("disney_plus_shows.csv")

In [3]:
desdf.show(truncate=False)

+----------+------------------------------------------+----+------+---------+-----+-----------+-----------------+-------+---------------------------------------------------------------+-----------------------------------------------+----+----+----------------+------------------------------+-------------------------------------------------------------+---------+-----------+----------+
|imdb_id   |title                                     |_c2 |type  |rated    |year |released_at|added_at         |runtime|genre                                                          |director                                       |_c11|_c12|language        |country                       |awards                                                       |metascore|imdb_rating|imdb_votes|
+----------+------------------------------------------+----+------+---------+-----+-----------+-----------------+-------+---------------------------------------------------------------+-----------------------------------------

In [4]:
desdf.printSchema()

root
 |-- imdb_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- type: string (nullable = true)
 |-- rated: string (nullable = true)
 |-- year: string (nullable = true)
 |-- released_at: string (nullable = true)
 |-- added_at: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- director: string (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- language: string (nullable = true)
 |-- country: string (nullable = true)
 |-- awards: string (nullable = true)
 |-- metascore: string (nullable = true)
 |-- imdb_rating: string (nullable = true)
 |-- imdb_votes: string (nullable = true)



In [5]:
desdf1 = desdf.drop("_c2").drop("year").drop("_c11").drop("_c12")

In [6]:
desdf1.printSchema()

root
 |-- imdb_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- type: string (nullable = true)
 |-- rated: string (nullable = true)
 |-- released_at: string (nullable = true)
 |-- added_at: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- director: string (nullable = true)
 |-- language: string (nullable = true)
 |-- country: string (nullable = true)
 |-- awards: string (nullable = true)
 |-- metascore: string (nullable = true)
 |-- imdb_rating: string (nullable = true)
 |-- imdb_votes: string (nullable = true)



In [7]:
desdf1.show()

+----------+--------------------+------+---------+-----------+-----------------+-------+--------------------+--------------------+----------------+--------------------+--------------------+---------+-----------+----------+
|   imdb_id|               title|  type|    rated|released_at|         added_at|runtime|               genre|            director|        language|             country|              awards|metascore|imdb_rating|imdb_votes|
+----------+--------------------+------+---------+-----------+-----------------+-------+--------------------+--------------------+----------------+--------------------+--------------------+---------+-----------+----------+
| tt0147800|10 Things I Hate ...| movie|    PG-13|  31-Mar-99|November 12, 2019| 97 min|Comedy, Drama, Ro...|          Gil Junger| English, French|                 USA|2 wins & 13 nomin...|       70|        7.3|  2,83,945|
| tt7019028|101 Dalmatian Street|series|      N/A|  25-Mar-19|February 28, 2020|    N/A|Animation, Comedy...

<h1>Handling N/A values</h1>

In [8]:
#replacing N/A values of string with unknown
desdf2= desdf1.replace("N/A", "Unknown", subset=['title','type','genre','director','language','country','awards'])

desdf3 = desdf2.replace("N/A","Unrated", subset=['rated'])
#replacing N/A values of int or float with 0
desdf4 = desdf3.replace("N/A", '0', subset=['imdb_votes','imdb_rating','metascore','runtime'])

In [9]:
desdf4.show()

+----------+--------------------+------+---------+-----------+-----------------+-------+--------------------+--------------------+----------------+--------------------+--------------------+---------+-----------+----------+
|   imdb_id|               title|  type|    rated|released_at|         added_at|runtime|               genre|            director|        language|             country|              awards|metascore|imdb_rating|imdb_votes|
+----------+--------------------+------+---------+-----------+-----------------+-------+--------------------+--------------------+----------------+--------------------+--------------------+---------+-----------+----------+
| tt0147800|10 Things I Hate ...| movie|    PG-13|  31-Mar-99|November 12, 2019| 97 min|Comedy, Drama, Ro...|          Gil Junger| English, French|                 USA|2 wins & 13 nomin...|       70|        7.3|  2,83,945|
| tt7019028|101 Dalmatian Street|series|  Unrated|  25-Mar-19|February 28, 2020|      0|Animation, Comedy...

<h1># Droping unnecessary Rows & Columns from table </h1>

In [10]:
desdf5 =desdf4.na.drop(subset=["imdb_id"])

In [11]:
desdf7= desdf5.drop("_c2").drop("year").drop("_c11").drop("_c12")

In [12]:
desdf7.printSchema()

root
 |-- imdb_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- type: string (nullable = true)
 |-- rated: string (nullable = true)
 |-- released_at: string (nullable = true)
 |-- added_at: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- director: string (nullable = true)
 |-- language: string (nullable = true)
 |-- country: string (nullable = true)
 |-- awards: string (nullable = true)
 |-- metascore: string (nullable = true)
 |-- imdb_rating: string (nullable = true)
 |-- imdb_votes: string (nullable = true)



In [13]:
desdf7.select("released_at").show()

+-----------+
|released_at|
+-----------+
|  31-Mar-99|
|  25-Mar-19|
|  27-Nov-96|
|  21-Jan-03|
|  22-Nov-00|
|  11-Dec-11|
|  20-Jul-55|
|  25-Nov-98|
|  10-Apr-20|
|  07-Apr-95|
|  11-Aug-95|
|  08-Mar-46|
|  23-Aug-02|
|  22-Jun-77|
|  09-Mar-18|
|  24-May-19|
|  25-Nov-92|
|  13-Aug-96|
|  20-May-94|
|  14-Sep-51|
+-----------+
only showing top 20 rows



<h1># Remove Duplicates</h1>

In [14]:
desdf8 =desdf7.dropDuplicates()

In [15]:
desdf8.printSchema()

root
 |-- imdb_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- type: string (nullable = true)
 |-- rated: string (nullable = true)
 |-- released_at: string (nullable = true)
 |-- added_at: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- director: string (nullable = true)
 |-- language: string (nullable = true)
 |-- country: string (nullable = true)
 |-- awards: string (nullable = true)
 |-- metascore: string (nullable = true)
 |-- imdb_rating: string (nullable = true)
 |-- imdb_votes: string (nullable = true)



In [16]:
desdf8.show()

+---------+--------------------+-------+---------+-----------+-----------------+-------+--------------------+--------------------+-------------------+-----------+--------------------+---------+-----------+----------+
|  imdb_id|               title|   type|    rated|released_at|         added_at|runtime|               genre|            director|           language|    country|              awards|metascore|imdb_rating|imdb_votes|
+---------+--------------------+-------+---------+-----------+-----------------+-------+--------------------+--------------------+-------------------+-----------+--------------------+---------+-----------+----------+
|tt0104437|Honey, I Blew Up ...|  movie|       PG|  17-Jul-92|November 12, 2019| 89 min|Adventure, Comedy...|      Randal Kleiser|            English|        USA|     10 nominations.|       50|        4.8|    39,775|
|tt7979492|        Kim Possible|  movie|     TV-G|  15-Feb-19|November 12, 2019| 86 min|Action, Adventure...|Zach Lipovsky, Ad...|  

<h1>Removing 'min' string from column and changing datatype to int</h1>

In [42]:
desdf8.printSchema()
desdf8.select("runtime").show()

root
 |-- imdb_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- type: string (nullable = true)
 |-- rated: string (nullable = true)
 |-- released_at: string (nullable = true)
 |-- added_at: string (nullable = true)
 |-- runtime: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- director: string (nullable = true)
 |-- language: string (nullable = true)
 |-- country: string (nullable = true)
 |-- awards: string (nullable = true)
 |-- metascore: string (nullable = true)
 |-- imdb_rating: string (nullable = true)
 |-- imdb_votes: string (nullable = true)

+-------+
|runtime|
+-------+
| 89 min|
| 86 min|
| 88 min|
| 88 min|
|130 min|
| 89 min|
| 90 min|
|128 min|
| 30 min|
| 99 min|
|105 min|
|      0|
| 80 min|
|  9 min|
|  4 min|
| 92 min|
| 36 min|
| 23 min|
|112 min|
| 20 min|
+-------+
only showing top 20 rows



In [17]:
from pyspark.sql.functions import regexp_extract, col, when, to_timestamp, to_date, regexp_replace
from pyspark.sql.types import IntegerType

desdf9 = desdf8.withColumn(
    "runtime", 
    when(
        regexp_extract(col("runtime"), r"(\d+)\s*min", 1) !="",
        regexp_extract(col("runtime"), r"(\d+)\s*min", 1).cast(IntegerType())
    ).otherwise(0)
)



In [37]:
desdf9.printSchema()
desdf9.select("runtime").show()

root
 |-- imdb_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- type: string (nullable = true)
 |-- rated: string (nullable = true)
 |-- released_at: string (nullable = true)
 |-- added_at: string (nullable = true)
 |-- runtime: integer (nullable = true)
 |-- genre: string (nullable = true)
 |-- director: string (nullable = true)
 |-- language: string (nullable = true)
 |-- country: string (nullable = true)
 |-- awards: string (nullable = true)
 |-- metascore: string (nullable = true)
 |-- imdb_rating: string (nullable = true)
 |-- imdb_votes: string (nullable = true)

+-------+
|runtime|
+-------+
|     89|
|     86|
|     88|
|     88|
|    130|
|     89|
|     90|
|    128|
|     30|
|     99|
|    105|
|      0|
|     80|
|      9|
|      4|
|     92|
|     36|
|     23|
|    112|
|     20|
+-------+
only showing top 20 rows



<h1># Changing datatypes to original datatypes</h1>

In [40]:
desdf9.printSchema()

root
 |-- imdb_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- type: string (nullable = true)
 |-- rated: string (nullable = true)
 |-- released_at: string (nullable = true)
 |-- added_at: string (nullable = true)
 |-- runtime: integer (nullable = true)
 |-- genre: string (nullable = true)
 |-- director: string (nullable = true)
 |-- language: string (nullable = true)
 |-- country: string (nullable = true)
 |-- awards: string (nullable = true)
 |-- metascore: string (nullable = true)
 |-- imdb_rating: string (nullable = true)
 |-- imdb_votes: string (nullable = true)



In [19]:
cast_columns = { 
    "metascore" : "int",
    "imdb_rating" : "float"}

temp_df = desdf9

for col_name, new_type in cast_columns.items():
    temp_df = temp_df.withColumn(col_name,col(col_name).cast(new_type))

desdf10 = temp_df



In [41]:
desdf10.printSchema()

root
 |-- imdb_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- type: string (nullable = true)
 |-- rated: string (nullable = true)
 |-- released_at: string (nullable = true)
 |-- added_at: string (nullable = true)
 |-- runtime: integer (nullable = true)
 |-- genre: string (nullable = true)
 |-- director: string (nullable = true)
 |-- language: string (nullable = true)
 |-- country: string (nullable = true)
 |-- awards: string (nullable = true)
 |-- metascore: integer (nullable = true)
 |-- imdb_rating: float (nullable = true)
 |-- imdb_votes: string (nullable = true)



<h1>Removing Comma from votes and changing Datatype</h1>

In [46]:
desdf10.printSchema()
desdf10.select("imdb_votes").show()

root
 |-- imdb_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- type: string (nullable = true)
 |-- rated: string (nullable = true)
 |-- released_at: string (nullable = true)
 |-- added_at: string (nullable = true)
 |-- runtime: integer (nullable = true)
 |-- genre: string (nullable = true)
 |-- director: string (nullable = true)
 |-- language: string (nullable = true)
 |-- country: string (nullable = true)
 |-- awards: string (nullable = true)
 |-- metascore: integer (nullable = true)
 |-- imdb_rating: float (nullable = true)
 |-- imdb_votes: string (nullable = true)

+----------+
|imdb_votes|
+----------+
|    39,775|
|     2,030|
|     1,166|
|     2,629|
|  7,30,917|
|     3,952|
|     1,736|
|  2,04,537|
|     1,143|
|       390|
|    63,682|
|       255|
|    29,395|
|     2,028|
|    11,759|
|  7,79,728|
|       283|
|    27,880|
|    55,628|
|     2,643|
+----------+
only showing top 20 rows



In [21]:
desdf11 = desdf10.withColumn(
    "imdb_votes",
    regexp_replace(col("imdb_votes"), ",", "").cast(IntegerType())
)


In [47]:
desdf11.printSchema()
desdf11.select("imdb_votes").show()

root
 |-- imdb_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- type: string (nullable = true)
 |-- rated: string (nullable = true)
 |-- released_at: string (nullable = true)
 |-- added_at: string (nullable = true)
 |-- runtime: integer (nullable = true)
 |-- genre: string (nullable = true)
 |-- director: string (nullable = true)
 |-- language: string (nullable = true)
 |-- country: string (nullable = true)
 |-- awards: string (nullable = true)
 |-- metascore: integer (nullable = true)
 |-- imdb_rating: float (nullable = true)
 |-- imdb_votes: integer (nullable = true)

+----------+
|imdb_votes|
+----------+
|     39775|
|      2030|
|      1166|
|      2629|
|    730917|
|      3952|
|      1736|
|    204537|
|      1143|
|       390|
|     63682|
|       255|
|     29395|
|      2028|
|     11759|
|    779728|
|       283|
|     27880|
|     55628|
|      2643|
+----------+
only showing top 20 rows



<h1>Consitent data throughout the column</h1>

In [51]:
desdf11.select("rated").show(40)

+---------+
|    rated|
+---------+
|       PG|
|     TV-G|
|    TV-PG|
|        G|
|    PG-13|
|       PG|
|     TV-G|
|       PG|
|     TV-Y|
|       PG|
|       PG|
|  Unrated|
|       PG|
| Approved|
|Not Rated|
|        G|
|  Unrated|
|     TV-G|
|        G|
|    TV-Y7|
|    PG-13|
|     TV-Y|
|    TV-Y7|
|       PG|
|       PG|
|        G|
|  UNRATED|
|  Unrated|
|     TV-G|
|  Unrated|
|Not Rated|
|    TV-Y7|
|  Unrated|
|       PG|
|  Unrated|
|        G|
|        G|
| Approved|
|        G|
|    TV-PG|
+---------+
only showing top 40 rows



In [48]:
#Changing all UNRATED values from uppercase to lowercase
desdf12 =desdf11.replace("UNRATED","Unrated",subset=["rated"])

#Changing all APPROVED values from uppercase to lowercase
desdf13 =desdf12.replace("APPROVED","Approved", subset=["rated"])

In [53]:
desdf13.select("rated").show(40)

+---------+
|    rated|
+---------+
|       PG|
|     TV-G|
|    TV-PG|
|        G|
|    PG-13|
|       PG|
|     TV-G|
|       PG|
|     TV-Y|
|       PG|
|       PG|
|  Unrated|
|       PG|
| Approved|
|Not Rated|
|        G|
|  Unrated|
|     TV-G|
|        G|
|    TV-Y7|
|    PG-13|
|     TV-Y|
|    TV-Y7|
|       PG|
|       PG|
|        G|
|  Unrated|
|  Unrated|
|     TV-G|
|  Unrated|
|Not Rated|
|    TV-Y7|
|  Unrated|
|       PG|
|  Unrated|
|        G|
|        G|
| Approved|
|        G|
|    TV-PG|
+---------+
only showing top 40 rows



In [54]:
# Changing datatype to timestamp
desdf15 = desdf13.withColumn("added_at", to_timestamp(col("added_at"), "MMMM d, yyyy"))

#changing datatype to date
desdf16 = desdf15.withColumn("added_at", to_date(col("added_at")))

In [58]:
desdf15.select("added_at").show()
desdf16.select("added_at").show()

+-------------------+
|           added_at|
+-------------------+
|2019-11-12 00:00:00|
|2019-11-12 00:00:00|
|2019-11-12 00:00:00|
|2019-11-12 00:00:00|
|2019-11-12 00:00:00|
|2019-11-12 00:00:00|
|2019-11-12 00:00:00|
|2020-01-08 00:00:00|
|2019-11-12 00:00:00|
|2019-11-12 00:00:00|
|2019-11-12 00:00:00|
|2019-11-12 00:00:00|
|2019-11-12 00:00:00|
|2019-11-12 00:00:00|
|2019-11-12 00:00:00|
|2019-11-12 00:00:00|
|2020-01-01 00:00:00|
|2019-11-12 00:00:00|
|2019-11-12 00:00:00|
|2019-11-12 00:00:00|
+-------------------+
only showing top 20 rows

+----------+
|  added_at|
+----------+
|2019-11-12|
|2019-11-12|
|2019-11-12|
|2019-11-12|
|2019-11-12|
|2019-11-12|
|2019-11-12|
|2020-01-08|
|2019-11-12|
|2019-11-12|
|2019-11-12|
|2019-11-12|
|2019-11-12|
|2019-11-12|
|2019-11-12|
|2019-11-12|
|2020-01-01|
|2019-11-12|
|2019-11-12|
|2019-11-12|
+----------+
only showing top 20 rows



In [56]:
desdf15.printSchema()
desdf16.printSchema()

root
 |-- imdb_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- type: string (nullable = true)
 |-- rated: string (nullable = true)
 |-- released_at: string (nullable = true)
 |-- added_at: timestamp (nullable = true)
 |-- runtime: integer (nullable = true)
 |-- genre: string (nullable = true)
 |-- director: string (nullable = true)
 |-- language: string (nullable = true)
 |-- country: string (nullable = true)
 |-- awards: string (nullable = true)
 |-- metascore: integer (nullable = true)
 |-- imdb_rating: float (nullable = true)
 |-- imdb_votes: integer (nullable = true)

root
 |-- imdb_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- type: string (nullable = true)
 |-- rated: string (nullable = true)
 |-- released_at: string (nullable = true)
 |-- added_at: date (nullable = true)
 |-- runtime: integer (nullable = true)
 |-- genre: string (nullable = true)
 |-- director: string (nullable = true)
 |-- language: string (nullable = true)
 |-- c

<h1>Seperating Awards to Wins, Oscars, Nominations, Golden_gloge_nominations to seperate Columns </h1>

In [27]:
desdf17 = desdf16.withColumn("wins", regexp_extract(col("awards"), r'(\d+)\s+win', 1).cast(IntegerType())) \
    .withColumn("nominations", regexp_extract(col("awards"), r'(\d+)\s+nomination', 1).cast(IntegerType())) \
    .withColumn("oscar_wins", regexp_extract(col("awards"), r'Won\s+(\d+)\s+Oscar', 1).cast(IntegerType())) \
    .withColumn("oscar_nominations", regexp_extract(col("awards"), r'Nominated for\s+(\d+)\s+Oscar', 1).cast(IntegerType())) \
    .withColumn("golden_globe_nominations", regexp_extract(col("awards"), r'Nominated for\s+(\d+)\s+Golden Globe', 1).cast(IntegerType())) \
    .fillna({
        "wins": 0,
        "nominations": 0,
        "oscar_wins": 0,
        "oscar_nominations": 0,
        "golden_globe_nominations": 0
    })

In [28]:
desdf17.printSchema()

root
 |-- imdb_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- type: string (nullable = true)
 |-- rated: string (nullable = true)
 |-- released_at: string (nullable = true)
 |-- added_at: date (nullable = true)
 |-- runtime: integer (nullable = true)
 |-- genre: string (nullable = true)
 |-- director: string (nullable = true)
 |-- language: string (nullable = true)
 |-- country: string (nullable = true)
 |-- awards: string (nullable = true)
 |-- metascore: integer (nullable = true)
 |-- imdb_rating: float (nullable = true)
 |-- imdb_votes: integer (nullable = true)
 |-- wins: integer (nullable = false)
 |-- nominations: integer (nullable = false)
 |-- oscar_wins: integer (nullable = false)
 |-- oscar_nominations: integer (nullable = false)
 |-- golden_globe_nominations: integer (nullable = false)



In [29]:
desdf17.select("awards","wins","nominations","oscar_wins","oscar_nominations","golden_globe_nominations").show()

+--------------------+----+-----------+----------+-----------------+------------------------+
|              awards|wins|nominations|oscar_wins|oscar_nominations|golden_globe_nominations|
+--------------------+----+-----------+----------+-----------------+------------------------+
|     10 nominations.|   0|         10|         0|                0|                       0|
|             Unknown|   0|          0|         0|                0|                       0|
|      3 nominations.|   0|          3|         0|                0|                       0|
|1 win & 1 nominat...|   1|          1|         0|                0|                       0|
|Nominated for 1 O...|  20|         62|         0|                1|                       0|
|      2 nominations.|   0|          2|         0|                0|                       0|
|       1 nomination.|   0|          1|         0|                0|                       0|
|7 wins & 32 nomin...|   7|         32|         0|          

In [59]:
#Replacing N/A values to None
desdf21 = desdf17.withColumn(
    "released_at",
    when(col("released_at").isin("N/A"), None).otherwise(col("released_at"))
)

In [32]:
desdf21.show()

+---------+--------------------+-------+---------+-----------+----------+-------+--------------------+--------------------+-------------------+-----------+--------------------+---------+-----------+----------+----+-----------+----------+-----------------+------------------------+
|  imdb_id|               title|   type|    rated|released_at|  added_at|runtime|               genre|            director|           language|    country|              awards|metascore|imdb_rating|imdb_votes|wins|nominations|oscar_wins|oscar_nominations|golden_globe_nominations|
+---------+--------------------+-------+---------+-----------+----------+-------+--------------------+--------------------+-------------------+-----------+--------------------+---------+-----------+----------+----+-----------+----------+-----------------+------------------------+
|tt0104437|Honey, I Blew Up ...|  movie|       PG|  17-Jul-92|2019-11-12|     89|Adventure, Comedy...|      Randal Kleiser|            English|        USA|  

In [33]:
desdf22 = desdf21.drop("awards")

In [34]:
desdf22.printSchema()

root
 |-- imdb_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- type: string (nullable = true)
 |-- rated: string (nullable = true)
 |-- released_at: string (nullable = true)
 |-- added_at: date (nullable = true)
 |-- runtime: integer (nullable = true)
 |-- genre: string (nullable = true)
 |-- director: string (nullable = true)
 |-- language: string (nullable = true)
 |-- country: string (nullable = true)
 |-- metascore: integer (nullable = true)
 |-- imdb_rating: float (nullable = true)
 |-- imdb_votes: integer (nullable = true)
 |-- wins: integer (nullable = false)
 |-- nominations: integer (nullable = false)
 |-- oscar_wins: integer (nullable = false)
 |-- oscar_nominations: integer (nullable = false)
 |-- golden_globe_nominations: integer (nullable = false)



In [36]:
desdf22.coalesce(1).write.csv("C:/Users/AMSPATIL/OneDrive - Capgemini/Desktop/cleaned_38",header=True ,mode="overwrite")